In [ ]:
using Pkg
Pkg.activate(".")
using SymPy

# Afterclass wrap up
The last example problem from class was based on Greenwood P6-9, with some modifications to be a 2-dof problem.

The work here is just on the part of the problem that we got through during class

In [ ]:
g, l, m, J, r, t = symbols("g l m J r t", real=true, positive=true)
ϕ = SymFunction("phi", real=true)(t)
θ = SymFunction("theta", real=true)(t)
p1 = SymFunction("p_1", real=true)(t)
p2 = SymFunction("p_2", real=true)(t)

ihat = [1, 0, 0]
jhat = [0, 1, 0]
khat = [0, 0, 1]
Dot(a, b) = sum(a .* b);

In [ ]:
R = r*(sin(ϕ)*ihat - cos(ϕ)*jhat) + l*(sin(θ)*ihat - cos(θ)*jhat)

In [ ]:
V = -m*g*Dot(R,jhat)

In [ ]:
T = 1//2*J*diff(ϕ,t)^2 + 1//2*m*Dot(diff.(R,t),diff.(R,t)) |> simplify

In [ ]:
L = T-V

In [ ]:
eq1 = -p1 + diff(L, diff(ϕ,t)) |> simplify

In [ ]:
eq2 = -p2 + diff(L, diff(θ, t)) |> simplify

In [ ]:
sol_p = solve([eq1, eq2], [diff(ϕ,t), diff(θ,t)])